In [1]:
# ruff: noqa: F401, ANN201

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import duckdb
import hvplot.polars
import numpy as np
import polars as pl

In [5]:
from tsdb_benchmarks.settings import SETTINGS

In [6]:
db = duckdb.connect(SETTINGS.results_directory / "results.db")

In [7]:
db.execute("select * from benchmark where finished_at is not null and deleted_at is null order by started_at").pl()

id,suite,db,operation,started_at,finished_at,deleted_at,notes
i32,str,str,str,datetime[μs],datetime[μs],datetime[μs],str
2,"""rtabench""","""monetdb""","""populate""",2025-07-17 13:42:29.301100,2025-07-17 13:52:54.675880,null,null
3,"""rtabench""","""monetdb""","""run""",2025-07-17 13:56:43.578373,2025-07-17 13:57:02.997623,null,null
4,"""rtabench""","""monetdb""","""run""",2025-07-17 13:57:34.648001,2025-07-17 13:57:52.085646,null,null
5,"""rtabench""","""monetdb""","""run""",2025-07-17 13:57:56.860219,2025-07-17 13:58:14.309203,null,null
6,"""rtabench""","""monetdb""","""run""",2025-07-17 14:02:14.373337,2025-07-17 14:02:49.868958,null,null
…,…,…,…,…,…,…,…
16,"""rtabench""","""timescaledb""","""populate""",2025-07-17 14:55:56.925806,2025-07-17 15:05:16.051233,null,null
17,"""rtabench""","""timescaledb""","""run""",2025-07-17 15:05:48.628531,2025-07-17 15:06:20.082169,null,null
18,"""rtabench""","""timescaledb""","""run""",2025-07-17 15:07:13.192146,2025-07-17 15:07:13.567165,null,null


In [21]:
def plot_queries(id: int):
    df = db.execute(
        """

    with filtered as (
        select *
        from event
        where benchmark_id = $id
        and name like 'query_%'
    ),
    starts as (
        select name, time as start_time
        from filtered
        where type = 'start'
    ),
    ends as (
        select name, time as end_time
        from filtered
        where type = 'end'
    )
    select
        substring(starts.name, 7, 4) as name,
        starts.start_time,
        ends.end_time,
        epoch(ends.end_time - starts.start_time) as duration
    from starts
    join ends using(name)
    order by starts.start_time


        """,
        {"id": id},
    ).pl()

    return df.hvplot(kind="bar", x="name", y="duration", fill_alpha=0.5, width=1200, grid=True)


plot_queries(14) * plot_queries(20)

:Overlay
   .Bars.I  :Bars   [name]   (duration)
   .Bars.II :Bars   [name]   (duration)

In [9]:
def plot_metrics(id: int, prefix: str | None = None):
    df = db.execute(
        """
            select
                epoch(time - (select min(time) from metric where benchmark_id = $id)) as time,
                mem_mb, disk_mb, cpu_percent
            from metric where benchmark_id = $id
            order by time
            """,
        {"id": id},
    ).pl()

    if prefix:
        df = df.select("time", pl.all().exclude("time").name.prefix(prefix))

    return df.hvplot(x="time", grid=True, width=900)


plot_metrics(7, "monetdb_") * plot_metrics(14, "clickhouse_")

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [time]   (value)

In [10]:
plot_metrics(2, "monetdb_") * plot_metrics(12, "clickhouse_")

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [time]   (value)

In [11]:
plot_metrics(12, "clickhouse_")

:NdOverlay   [Variable]
   :Curve   [time]   (value)